In [1]:
from pyspark.sql import SparkSession, Row


In [2]:
# SparkSession => entry to the DataFrame API
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
spark

In [18]:
# splits the lines and returns DataFrame
def parser(line):
    line = line.split(",")
    return Row(ID=int(line[0]),
               name=str(line[1]),
               age=int(line[2]),
               num_friends=int(line[3])
              )


In [22]:
# read the file as RDD
lines = spark.sparkContext.textFile("fakefriends.csv")
lines.take(5)


['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21']

In [28]:
# parses input file and returns rows
people = lines.map(parser)
people.take(5)


[Row(ID=0, name='Will', age=33, num_friends=385),
 Row(ID=1, name='Jean-Luc', age=26, num_friends=2),
 Row(ID=2, name='Hugh', age=55, num_friends=221),
 Row(ID=3, name='Deanna', age=40, num_friends=465),
 Row(ID=4, name='Quark', age=68, num_friends=21)]

In [31]:
# transforms RDD into DataFrame, store it in the cache, create temp view for SQL
df = spark.createDataFrame(people).cache()
df.createOrReplaceTempView("people")


In [32]:
teenagers = spark.sql(
    """
        select *
        from people
        where age >= 13 and age <= 19
    """)


In [33]:
teenagers.show()

+---+-------+---+-----------+
| ID|   name|age|num_friends|
+---+-------+---+-----------+
| 21|  Miles| 19|        268|
| 52|Beverly| 19|        269|
| 54|  Brunt| 19|          5|
|106|Beverly| 18|        499|
|115|  Dukat| 18|        397|
|133|  Quark| 19|        265|
|136|   Will| 19|        335|
|225|   Elim| 19|        106|
|304|   Will| 19|        404|
|341|   Data| 18|        326|
|366|  Keiko| 19|        119|
|373|  Quark| 19|        272|
|377|Beverly| 18|        418|
|404| Kasidy| 18|         24|
|409|    Nog| 19|        267|
|439|   Data| 18|        417|
|444|  Keiko| 18|        472|
|492|  Dukat| 19|         36|
|494| Kasidy| 18|        194|
+---+-------+---+-----------+



In [39]:
# calculating num of people per age
df.groupBy("age").count().orderBy("age").show()

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
| 28|   10|
| 29|   12|
| 30|   11|
| 31|    8|
| 32|   11|
| 33|   12|
| 34|    6|
| 35|    8|
| 36|   10|
| 37|    9|
+---+-----+
only showing top 20 rows

